# **PONTIFÍCIA UNIVERSIDADE CATÓLICA DE MINAS GERAIS - NÚCLEO DE EDUCAÇÃO A DISTÂNCIA**

### Pós-graduação Lato Sensu em Ciência de Dados e Big Data


### **3. Análise e Exploração dos dados**

**Carrega as bibliotecas**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import percentile
import matplotlib

# Scaling variables
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer

np.set_printoptions(precision=4, suppress=True)  # Formatando impressão de Array Numpy
pd.set_option('display.float_format', lambda x: '%.6f' % x)  # Formatando impressão de numeros no Pandas

%matplotlib inline

In [ ]:
!pip install sweetviz   ## Instalar a Biblioteca de Visualização de Dados Sweetviz

In [ ]:
import sweetviz as sv

**Carrega os dados**

In [ ]:
# Ao carregar a base já ajustada, manter os tipos das colunas conforme definição na etapa de Tratamento (Notebook_Rais02_Tratamento)
df_rais2019CE = pd.read_csv('./BD_Rais_2019_CE_02.csv', dtype={ "id_municipio":                "string",
                                                               "qtde_vinculos_ativos":        "int64",
                                                               "qtde_vinculos_clt":          "int64",
                                                               "qtde_vinculos_estatutarios": "int64",
                                                               "natureza_juridica":           "string",
                                                               "tamanho_estabelecimento":    "int64",
                                                               "indicador_cei_vinculado":    "int64",
                                                               "indicador_pat":              "int64",
                                                               "indicador_simples":          "int64",
                                                               "indicador_rais_negativa":    "int64",
                                                               "indicador_atividade_ano":    "int64",
                                                               "cnae_2":                     "string",
                                                               "cnae_2_subclasse":           "string",
                                                               "subsetor_ibge":              "string",
                                                               "cep_estabelecimento":        "string"})


**Fazer análise com a biblioteca SWEETVIZ**

In [ ]:
# Analizando o dataset
rais_report = sv.analyze(df_rais2019CE)

#Mostrando o Relatório
rais_report.show_html('Rais.html')

                                             |          | [  0%]   00:00 -> (? left)

Report Rais.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
rais_report.show_notebook()

In [ ]:
# Eliminar os registros com RAIS Negativa 
filtro = df_rais2019CE.loc[df_rais2019CE['indicador_rais_negativa'] == 1]  
df_rais2019CE = df_rais2019CE.drop(filtro.index)

# Os campos 'indicador_rais_negativa" e "indicador_atividade_ano" tem valores constantes (0 e 1, respectivamente) 
# e, portanto, podem ser descartados
df_rais2019CE = df_rais2019CE.drop(columns=['indicador_rais_negativa', 'indicador_atividade_ano'])

display(df_rais2019CE.info())
df_rais2019CE.shape

In [ ]:
# Removendo Colunas indicador_cei_vinculado e indicador_pat
df_rais2019CE = df_rais2019CE.drop(columns=['indicador_cei_vinculado','indicador_pat'])
display(df_rais2019CE.info())
display(df_rais2019CE.shape)

### Realizar One-Hot-Encoding 

In [ ]:
## Criar as colunas dummies a partir da coluna 'natureza juridica' agrupadas pelo 1ª digito
dum_nat_jur = pd.get_dummies(df_rais2019CE['natureza_juridica'].str.slice(0,1), prefix='grupo_nj')

## Acrescentar novas colunas ao Dataset
df_rais2019CE = pd.concat([df_rais2019CE, dum_nat_jur], axis = 1)

In [ ]:
df_CNAE = pd.read_csv('./BD_CNAE2.csv',dtype={ "classe_id": "string"})

## Acrescentar a coluna 'cnae_2_secao' no dataset, baseada na coluna 'cnae_2'
for linha, coluna in df_rais2019CE.iterrows(): 
    cnae_classe = coluna["cnae_2"]
    df_secao = df_CNAE.loc[df_CNAE['classe_id'] == cnae_classe]
    secao = df_secao['secao_id'].unique()
    df_rais2019CE.at[linha,'cnae_2_secao'] = secao[0]

In [ ]:
## Criar as colunas dummies a partir da coluna 'CNAE_2' agrupadas por Seçao da CNAE
dum_cnae = pd.get_dummies(df_rais2019CE['cnae_2_secao'], prefix='cnae_2_secao')

## Acrescentar novas colunas ao Dataset
df_rais2019CE = pd.concat([df_rais2019CE, dum_cnae], axis = 1)

In [ ]:
## Criar as colunas dummies a partir da coluna 'subsetor_ibge' 
dum_ibge = pd.get_dummies(df_rais2019CE['subsetor_ibge'], prefix='sub_ibge')

## Acrescentar novas colunas ao Dataset
df_rais2019CE = pd.concat([df_rais2019CE, dum_ibge], axis = 1)

In [ ]:
## Salvar o novo dataset após o tratamento realizado nesta etapa
df_rais2019CE.to_csv('BD_Rais_2019_CE_03.csv', index=False)

# **FIM DO NOTEBOOK 03 AED**